In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [7]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [9]:
x=scaler.fit_transform(x)

In [11]:
x.shape

(768, 8)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [89]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [22]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid')) # Changed to 1 unit for binary classification
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5158 - loss: 0.7104 - val_accuracy: 0.5844 - val_loss: 0.6584
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6175 - loss: 0.6510 - val_accuracy: 0.6948 - val_loss: 0.6090
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6939 - loss: 0.5951 - val_accuracy: 0.7403 - val_loss: 0.5764
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7523 - loss: 0.5667 - val_accuracy: 0.7662 - val_loss: 0.5518
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7452 - loss: 0.5542 - val_accuracy: 0.7597 - val_loss: 0.5332
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7403 - loss: 0.5394 - val_accuracy: 0.7597 - val_loss: 0.5183
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7564 - loss: 0.5400 - val_accuracy: 0.7532 - val_loss: 0.5078
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7617 - loss: 0.5105 - val_accuracy: 0.7597 

In [24]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
#how to select appropriate optimizer
#number of nodes in a layer
#how to select the number of hidden layers
#all in one model

**Optimizer Selection**

In [26]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [39]:
def build_model(hp):

  model=Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [40]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [41]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [42]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [43]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6884 - loss: 0.5730 - val_accuracy: 0.7662 - val_loss: 0.5264
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6961 - loss: 0.5540 - val_accuracy: 0.7727 - val_loss: 0.5127
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7280 - loss: 0.5316 - val_accuracy: 0.7792 - val_loss: 0.5011
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7450 - loss: 0.5135 - val_accuracy: 0.7857 - val_loss: 0.4932
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7385 - loss: 0.5005 - val_accuracy: 0.7792 - val_loss: 0.4883
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7385 - loss: 0.4959 - val_accuracy: 0.7922 - val_loss: 0.4843
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7528 - loss: 0.5085 - val_accuracy: 0.7922 - val_loss: 0.4807
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7466 - loss: 0.4792 - val_accuracy: 0.

**Right number of neurons in the layer**

In [64]:
def build_model(hp):

  model=Sequential()

  units=hp.Int('units',min_value=8,max_value=128)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [65]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,
                      directory='mydir',
                      project_name='Kapil')

Reloading Tuner from mydir/Kapil/tuner0.json


In [66]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [67]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [68]:
model=tuner.get_best_models(num_models=1)[0]

In [69]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7496 - loss: 0.4943 - val_accuracy: 0.7987 - val_loss: 0.4783
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7885 - loss: 0.4573 - val_accuracy: 0.7922 - val_loss: 0.4745
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7510 - loss: 0.4646 - val_accuracy: 0.7987 - val_loss: 0.4734
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7531 - loss: 0.4717 - val_accuracy: 0.7987 - val_loss: 0.4711
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7614 - loss: 0.4513 - val_accuracy: 0.7987 - val_loss: 0.4709
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7810 - loss: 0.4477 - val_accuracy: 0.8182 - val_loss: 0.4697
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7726 - loss: 0.4708 - val_accuracy: 0.8052 - val_loss: 0.4691
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7872 - loss: 0.4444 - val_accuracy: 0.80

**how to select the number of hidden layers**

In [70]:
def build_model(hp):

  model=Sequential()

  model.add(Dense(88,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layer',min_value=1,max_value=10)):

    model.add(Dense(88,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [73]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',
                      max_trials=3,
                      directory='mydir',
                      project_name='num_layer')

In [74]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 20s


In [75]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 4}

In [76]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [77]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7761 - loss: 0.4737 - val_accuracy: 0.7597 - val_loss: 0.5011
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7955 - loss: 0.3945 - val_accuracy: 0.7662 - val_loss: 0.4981
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8083 - loss: 0.4136 - val_accuracy: 0.7597 - val_loss: 0.5015
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8086 - loss: 0.4103 - val_accuracy: 0.8052 - val_loss: 0.4914
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8011 - loss: 0.4212 - val_accuracy: 0.7857 - val_loss: 0.5308
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8354 - loss: 0.3765 - val_accuracy: 0.7662 - val_loss: 0.5302
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8347 - loss: 0.3511 - val_accuracy: 0.7922 - val_loss: 0.5335
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8308 - loss: 0.3698 - val_accuracy: 0

# **All in One**

In [91]:
def build_model(hp):

  model=Sequential()
  counter=0
  for i in range(hp.Int('num_layer',min_value=1,max_value=10)):

    if counter==0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128),
      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
      input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128),
      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
    counter +=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [92]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      directory='mydir',
                      project_name='final1')

In [93]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 57s


In [95]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 2,
 'units0': 20,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'adam',
 'units1': 122,
 'activation1': 'sigmoid',
 'units2': 45,
 'activation2': 'relu',
 'units3': 100,
 'activation3': 'tanh',
 'units4': 100,
 'activation4': 'relu',
 'units5': 71,
 'activation5': 'sigmoid',
 'units6': 32,
 'activation6': 'tanh',
 'units7': 80,
 'activation7': 'relu',
 'units8': 97,
 'activation8': 'sigmoid'}

In [96]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [97]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7219 - loss: 0.5296 - val_accuracy: 0.7792 - val_loss: 0.4888
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7543 - loss: 0.5013 - val_accuracy: 0.7792 - val_loss: 0.4755
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7504 - loss: 0.5003 - val_accuracy: 0.7727 - val_loss: 0.4698
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7678 - loss: 0.4803 - val_accuracy: 0.7662 - val_loss: 0.4675
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7495 - loss: 0.4906 - val_accuracy: 0.7662 - val_loss: 0.4651
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7842 - loss: 0.4651 - val_accuracy: 0.7792 - val_loss: 0.4620
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7189 - loss: 0.5110 - val_accuracy: 0.7857 - val_loss: 0.4629
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7534 - loss: 0.5020 - val_accuracy: 